In [1]:
# импорт необходимых библиотек
import pandas as pd
import re
from pymystem3 import Mystem
from IPython.display import display
pd.options.display.max_colwidth = 100
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r'C:\Users\Iya\Downloads\test_data.csv')
display(df.head())

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серы...
4,0,4,client,Ага


Проведем очистку и лемматизацию текста

In [3]:
m=Mystem()
def lemmatize(text):
   text = text.lower()
   lemm_text = "".join(m.lemmatize(text))
   cleared_text = re.sub(r'[^а-яА-ЯёЁ]', ' ', lemm_text)
   return " ".join(cleared_text.split())
df['lemm_text'] = df['text'].apply(lemmatize)

In [4]:
display(df.head())

,dlg_id,line_n,role,text,lemm_text
0,0,0,client,Алло,але
1,0,1,manager,Алло здравствуйте,але здравствовать
2,0,2,client,Добрый день,добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серы...,я звать ангелина компания диджитать бизнес звонить вы по повод продление лицензия а мы с серый у...
4,0,4,client,Ага,ага


Создадим список ключевых слов, по которым поставим метки текста. Понадобятся приветствия, прощания и представление себя

In [5]:
list_greetings=['привет','добрый день', 'день добрый', 'приветствовать', 'здравствовать', 'добрый утро', 
                'утро добрый', 'добрый вечер', 'вечер добрый', 'здрасте', 'салют', 'добрый ночь', 'ночь добрый']
list_goodbye=['прощать','до свидание']
list_introduce= ['звать', 'имя', 'мой имя']

In [6]:
def insights(text):             # функция для разметки текста по ключевым словам
    for i in list_greetings:
        if i in text:
            return 'greeting'
    for i in list_goodbye:
        if i in text:
            return 'goodbye'
    for i in list_introduce:
        if i in text:
            return 'introduce'           

Применим функцию к тексту. Метки будут в отдельной колонке `insights`

In [7]:
df['insights'] = df['lemm_text'].apply(insights)
display(df.head())

,dlg_id,line_n,role,text,lemm_text,insights
0,0,0,client,Алло,але,None
1,0,1,manager,Алло здравствуйте,але здравствовать,greeting
2,0,2,client,Добрый день,добрый день,greeting
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серы...,я звать ангелина компания диджитать бизнес звонить вы по повод продление лицензия а мы с серый у...,introduce
4,0,4,client,Ага,ага,None


In [8]:
display(df.tail())

,dlg_id,line_n,role,text,lemm_text,insights
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкретно продам,по повод виджет и с они уже обсуждать конкретно продавать,None
476,5,139,manager,Все я вам высылаю счет и с вами на связи если будут вопросы можете писать на ватсапе,все я вы высылать счет и с вы на связь если быть вопрос мочь писать на ватсапа,None
477,5,140,client,Спасибо спасибо,спасибо спасибо,None
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,да да тогда созваниваться ага спасибо вы давать,None
479,5,142,manager,Ну до свидания хорошего вечера,ну до свидание хороший вечер,goodbye


Напишем функцию для извлечения реплик менеджера  по меткам

In [9]:
def extract_repl(insights):                                              #insights указывать в кавычках
    df_cut = df[(df['role']=='manager')&(df['insights']==insights)]
    df_cut= df_cut.drop('lemm_text', axis =1)
    display(df_cut)

Реплики с приветствием – где менеджер поздоровался.

In [10]:
extract_repl('greeting')

,dlg_id,line_n,role,text,insights
1,0,1,manager,Алло здравствуйте,greeting
110,1,1,manager,Алло здравствуйте,greeting
166,2,2,manager,Алло здравствуйте,greeting
250,3,1,manager,Алло дмитрий добрый день,greeting


Реплики, где менеджер представил себя.

In [11]:
extract_repl('introduce')

,dlg_id,line_n,role,text,insights
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серы...,introduce
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила вн...,introduce
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы...,introduce
251,3,2,manager,Добрый меня максим зовут компания китобизнес удобно говорить,introduce


Реплики, где менеджер попрощался.

In [12]:
extract_repl('goodbye')

,dlg_id,line_n,role,text,insights
108,0,108,manager,Всего хорошего до свидания,goodbye
163,1,54,manager,До свидания,goodbye
335,4,33,manager,Во вторник все ну с вами да тогда до вторника до свидания,goodbye
479,5,142,manager,Ну до свидания хорошего вечера,goodbye


Напишем фунцкию, которая будет проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [13]:
def check_alldialogs(role):
    dlg_list=list(df['dlg_id'].unique())
    for i in dlg_list:
        df_ch = df[(df['dlg_id']==i)&(df['role']==role)]
        if len(df_ch[df_ch['insights']=='goodbye'])>=1 and len(df_ch[df_ch['insights']=='greeting'])>=1:
            print ('Диалог', i ,': Менеджер поздоровался и попрощался с клиентом')
        elif len(df_ch[df_ch['insights']=='goodbye'])>=1 and len(df_ch[df_ch['insights']=='greeting'])==0:
            print ('Диалог', i ,': Менеджер, вероятно, не поздоровался с клиентом') 
        elif len(df_ch[df_ch['insights']=='goodbye'])==0 and len(df_ch[df_ch['insights']=='greeting'])>=1:
            print ('Диалог', i ,': Менеджер, вероятно, не попрощался с клиентом')  
        elif len(df_ch[df_ch['insights']=='goodbye'])==0 and len(df_ch[df_ch['insights']=='greeting'])==0:
            print ('Диалог', i ,': Менеджер, вероятно, не поздоровался и не попрощался с клиентом')     

Проверим функцию на тексте

In [14]:
check_alldialogs('manager')

Диалог 0 : Менеджер поздоровался и попрощался с клиентом
Диалог 1 : Менеджер поздоровался и попрощался с клиентом
Диалог 2 : Менеджер, вероятно, не попрощался с клиентом
Диалог 3 : Менеджер, вероятно, не попрощался с клиентом
Диалог 4 : Менеджер, вероятно, не поздоровался с клиентом
Диалог 5 : Менеджер, вероятно, не поздоровался с клиентом


Проверим реплики менеджера, где он называл себя. Как правило пердставляют себя фразами: "Мое имя..." , "Меня зовут.."
Проверим в тексте какие слова использовались менеджерами. Ключевые слова: "имя" и "звать"

In [15]:
df[(df['role']=='manager')]['lemm_text'].str.contains('имя', regex=False, case=False, na=False).value_counts()

False    201
Name: lemm_text, dtype: int64

In [16]:
df[(df['role']=='manager')]['lemm_text'].str.contains('звать', regex=False, case=False, na=False).value_counts()

False    197
True       4
Name: lemm_text, dtype: int64

Найдены реплики только со словом "звать". По этому слову напишем функцию, которая будет искать соседние слова. Так как в речи возможно упортебление имени и до и после слова "звать", то будем извлекать по 1 слову справа и слева 

In [17]:
def names(role):
    df_intr=df[(df['role']==role)&(df['insights']=='introduce')]
    dlg_list=set(df_intr['dlg_id'].unique())
    for i in dlg_list:
        names= str(df_intr[df['dlg_id']==i]['lemm_text'])
        names_before=names.split("звать", 1)[0].split()[-1]
        names_after=names.split("звать", 1)[1].split()[0]
        print("Диалог", i, ": " , names_before, names_after)
    dlg_list_all=set(df['dlg_id'].unique())  
    diff = dlg_list_all- dlg_list    # поиск диалогов, в которых не найдено ключевое слово
    for j in diff:
        print('Диалог', j, ": " , "Имя не найдено")

In [18]:
names('manager')

Диалог 0 :  я ангелина
Диалог 1 :  я ангелина
Диалог 2 :  я ангелина
Диалог 3 :  максим компания
Диалог 4 :  Имя не найдено
Диалог 5 :  Имя не найдено


Напишем функцию для извлечения названия компании. Так как назвние компании может состоять из нескольких слов, извлечем из текста несколько слов после слова "компания"

In [19]:
def extract_company():
    def company(text):
        if 'компания ' in text:
            pattern =r'компан..\s+\w+\s+\w+\s+\w+'
            result = re.findall(pattern, text)
            return result
    df['company'] = df['text'].apply(company)
    df_comp= df[(df['role']=='manager')&(df['company'].notnull())&(df['line_n']<15)]
    df_comp=df_comp.drop(['lemm_text', 'insights'], axis=1)
    print('Название компании содержится в колонке', '\033[1m' + 'company' + '\033[0m')
    display(df_comp)

Проверим на тексте

In [20]:
extract_company()

Название компании содержится в колонке company


,dlg_id,line_n,role,text,company
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серы...,[компания диджитал бизнес звоним]
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила вн...,[компания диджитал бизнес звоню]
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы...,[компания диджитал бизнес звоню]
251,3,2,manager,Добрый меня максим зовут компания китобизнес удобно говорить,[компания китобизнес удобно говорить]
